In [60]:
import pandas as pd
import numpy as np
import datetime
from scipy import stats
from scipy.stats import t


file_path = 'sample.csv' # to use it in generate_predictions function

dataframe = pd.read_csv('sample.csv')

dataframe.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0005,B02510,02/12/2019 11:10:30 AM,02/12/2019 11:19:14 AM,80,112,NaN
1,HV0005,B02510,02/12/2019 11:35:27 AM,02/12/2019 11:54:29 AM,256,125,NaN
2,HV0003,B02867,02/12/2019 11:03:14 AM,02/12/2019 11:21:04 AM,210,155,1.0
3,HV0003,B02867,02/12/2019 11:12:12 AM,02/12/2019 11:24:40 AM,155,155,1.0
4,HV0003,B02867,02/12/2019 11:50:05 AM,02/12/2019 11:59:14 AM,155,91,NaN


In [50]:
dataframe.shape

(200000, 7)

### compute_trip_duration : Function to compute trip duration and add it to the dataframe

In [51]:
def compute_trip_duration(dataframe):
    dataframe['pickup_datetime'] = pd.to_datetime(dataframe['pickup_datetime'])
    dataframe['dropoff_datetime'] = pd.to_datetime(dataframe['dropoff_datetime'])
    #difference between pickup_datetime Column  and dropoff_datetime Column to find trip_duration Column.
    dataframe['trip_duration'] = ((dataframe['dropoff_datetime'] - dataframe['pickup_datetime']).dt.total_seconds() / 60)   
    return dataframe

#and I've divided the total seconds by 60 to convert it to minutes.

In [52]:
compute_trip_duration(dataframe)

dataframe.head(2)

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,trip_duration
0,HV0005,B02510,2019-02-12 11:10:30,2019-02-12 11:19:14,80,112,NaN,8.733333
1,HV0005,B02510,2019-02-12 11:35:27,2019-02-12 11:54:29,256,125,NaN,19.033333


In [53]:
#new one Column 
dataframe.shape

(200000, 8)

### time_features : Function to add hour of the day and day of the week to the dataframe

In [54]:
def time_features(dataframe):
    dataframe['pickup_datetime'] = pd.to_datetime(dataframe['pickup_datetime'])
    
    #new column containing integers representing the hour of the day(by using .dt.hour).
    dataframe['hour'] = dataframe['pickup_datetime'].dt.hour
    
    #new column containing integers representing the day of the week(by using .dt.dayofweek).
    dataframe['day_of_week'] = dataframe['pickup_datetime'].dt.dayofweek
    return dataframe

In [55]:
add_time_features(dataframe)

dataframe.head(2)

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,trip_duration,hour,day_of_week
0,HV0005,B02510,2019-02-12 11:10:30,2019-02-12 11:19:14,80,112,NaN,8.733333,11,1
1,HV0005,B02510,2019-02-12 11:35:27,2019-02-12 11:54:29,256,125,NaN,19.033333,11,1


In [56]:
#new two column 
dataframe.shape

(200000, 10)

### compute_predictions : Function to compute mean trip duration and margin of error.

In [57]:
def compute_predictions(dataframe, confidence_level=0.95):
    #it calculates the mean and standard error of the 'trip_duration' column using the agg method.
    predictions = dataframe.groupby(['PULocationID', 'DOLocationID', 'day_of_week', 'hour'])['trip_duration'].agg(['mean', 'sem'])
    #Calculates the margin of error for each group by multiplying the standard error ('sem') by 1.96.
    
    # Calculate the critical value (t-score) for the given confidence level
    t_score = t.ppf((1 + confidence_level) / 2, dataframe['trip_duration'].count() - 1)

    # Calculate the margin of error
    predictions['margin_of_error'] = t_score * predictions['sem']
    return predictions

def compute_predictions(dataframe, confidence_level=0.95):

    # Calculate the critical value (t-score) for the given confidence level
    t_score = t.ppf((1 + confidence_level) / 2, predictions['trip_duration'].count() - 1)


    return predictions

### generate_predictions : Function to read data file and generate predictions dataframe

In [58]:
def generate_predictions(file_path):
    taxi_data = pd.read_csv(file_path)
    taxi_data = compute_trip_duration(taxi_data)
    taxi_data = time_features(taxi_data)
    predictions = compute_predictions(taxi_data)
    return predictions


In [59]:
generate_predictions(file_path)

mean       sem  \
PULocationID DOLocationID day_of_week hour                        
1            1            4           6     26.966667       NaN   
             24           3           1     42.733333       NaN   
             79           1           10    54.933333       NaN   
             107          6           20    40.133333       NaN   
             137          2           19    43.816667       NaN   
...                                               ...       ...   
265          265          2           17     0.983333  0.033333   
                                      20    10.433333       NaN   
                                      23     8.941667  4.208333   
                          4           13    12.216667       NaN   
                          5           17     2.400000       NaN   

                                            margin_of_error  
PULocationID DOLocationID day_of_week hour                   
1            1            4           6                 NaN  
             24           3           1                 NaN  
             79           1           10                NaN  
             107          6           20                NaN  
             137          2           19                NaN  
...                                                     ...  
265          265          2           17           0.065333  
                                      20                NaN  
                                      23           8.248333  
                          4           13                NaN  
                          5           17                NaN  

[113483 rows x 3 columns]